In [ ]:
import os
import shutil
import pandas as pd
import datetime as dt
import calendar
import numpy as np
from sqlalchemy import create_engine, MetaData, desc
from sqlalchemy.sql import select, and_, or_, not_
from sqlalchemy.pool import NullPool
from sqlalchemy.sql import select

from settings.db import DO_URL

In [ ]:
def get_site_id(site_name, connection):
    print(site_name)
    site_id_response = connection.execute('SELECT id FROM public.sites where displayable_name = \'{0}\''.format(site_name))
    #print(site_name)
    #print('SELECT id FROM public.sites where displayable_name = \'{0}\''.format(site_name.capitalize()))
    #print(list(site_id_response))
    site_id = list(site_id_response)[0][0]
    return site_id


def get_sqlalchemy_table(table_name, connection):
    meta_data = MetaData(bind=connection, reflect=True)
    table = meta_data.tables[table_name]
    return table


def get_start_index(table, connection):
    start_index = None
    max_index_response = connection.execute('SELECT MAX(id) FROM public.{0}'.format(table))
    max_index = list(max_index_response)[0][0]
    start_index = 0 if max_index is None else int(max_index) + 1
    return start_index

In [ ]:
tariff_dir = 'data/green_tariff/'
green_tarrif_files =  [file for file in os.listdir(tariff_dir) if os.path.isfile(os.path.join(tariff_dir, file))]
if "desktop.ini" in green_tarrif_files:
    green_tarrif_files.remove('desktop.ini')

print(green_tarrif_files)

engine = create_engine(DO_URL, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

In [ ]:
def quarter_translation(quarter, year):
    values = {1: {'first_day': dt.date(year=year, month=1, day=1), 'last_day': dt.date(year=year, month=3, day=31)},
              2: {'first_day': dt.date(year=year, month=4, day=1), 'last_day': dt.date(year=year, month=6, day=30)},
              3: {'first_day': dt.date(year=year, month=7, day=1), 'last_day': dt.date(year=year, month=9, day=30)},
              4: {'first_day': dt.date(year=year, month=10, day=1), 'last_day': dt.date(year=year, month=12, day=31)}}
    return values[quarter]

In [ ]:
sheet_name = 'Sheet1'

file = green_tarrif_files[0]

print(file)

quarter = int(file.split('_')[2])
year = int(file.split('_')[-1].split('.')[0])

data = pd.read_excel(tariff_dir + file, sheet_name=sheet_name, header=0)
print(data)
data.columns = ['site', 'green_tariff']

with engine.connect() as connection:
    data['site'] = data['site'].apply(lambda x: get_site_id(x, connection))
    availability = quarter_translation(quarter, year)

    data['available_from'] = availability['first_day']
    data['available_to'] = availability['last_day']


    last_index = get_start_index('green_tariffs', connection)

data = data.sort_values(by=['site']).reset_index()

data['index'] = data.index + last_index + 1

print('ok!')
print(data)

In [ ]:
import io
import psycopg2


def connect():
    connection = psycopg2.connect(DO_URL)
    return connection


def translate_data(records):
    csv_file_like_object = io.StringIO()
    for index, record in records.iterrows():
            line = '|'.join(map(str, [record['index'], record['site'], record['available_from'], 
                             record['available_to'], record['green_tariff']])) + '\n'
            print(line)
            #logger.info(len(line.split('|')))
            csv_file_like_object.write(line)
    csv_file_like_object.seek(0)
    return csv_file_like_object


def copy_stringio(data_to_write, connection):
    try:
        with connection.cursor() as cursor:
            cursor.copy_from(data_to_write, 'green_tariffs', sep='|')
            connection.commit()  
    except Exception as e:
        connection.rollback()
        raise e
    finally:
        connection.close()

In [ ]:
csv_like_object = translate_data(data)

In [ ]:
connection = connect()
copy_stringio(csv_like_object, connection)
print('OK!')

shutil.move(tariff_dir + file, tariff_dir + '/uploaded/' + file)
print('OK!')